In [ ]:
from keras.applications import *
from keras.preprocessing.image import *
import h5py
import math
import gc
from keras.models import *
from keras.layers import *
from keras.optimizers import *
import numpy as np
from sklearn.utils import shuffle
import pandas as pd
from keras.callbacks import *
import matplotlib.pyplot as plt
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import StratifiedKFold

class LossHistory(Callback):
    def on_train_begin(self, logs={}):
        self.losses = []
        self.val_losses = []
        self.acc=[]
        self.val_acc=[]

    def on_epoch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))
        self.acc.append(logs.get('acc'))
        self.val_acc.append(logs.get('val_acc'))


nb_epoch = 500
np.random.seed(1000)
rootdir='model-final\\'
X_train = []
X_val = []
X_test = []
#-------------------------------------------------train
for filename in ['datax2crop_ResNet152.h5',"datax2crop_DenseNet161.h5", "datax2crop_Xception.h5"]: 
    with h5py.File(rootdir+filename, 'r') as h:
        X_train.append(np.array(h['train']))
        y_train = np.array(h['label'])
X_train = np.concatenate(X_train, axis=1)
#-------------------------------------------------test
for filename in ['testcrop_ResNet152.h5',"testcrop_DenseNet161.h5", "testcrop_Xception.h5"]: 
    with h5py.File(rootdir+'tmp\\'+filename, 'r') as h:
        X_test.append(np.array(h['test']))
X_test = np.concatenate(X_test, axis=1)

X_train, y_train = shuffle(X_train, y_train)
res=pd.DataFrame()
skf=list(StratifiedKFold(y_train, n_folds=5, shuffle=True, random_state=1000))     

dout=0.5
classes=97
y_train = np.eye(classes)[y_train]

### 顶层使用sigmoid会在一定epoch后出现骤降，所有样本全部预测为0类，而骤降前停止迭代预测效果较好(巧合？)

In [ ]:
rounds=37
i=0
train=skf[i][0]
input_tensor = Input(X_train.shape[1:])
x = input_tensor
x = Dropout(dout)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(classes, activation='softmax')(x)
model = Model(inputs=input_tensor, outputs=x)
model.compile(optimizer='adadelta',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
print("Fold", i)

model.fit(X_train[train], y_train[train], batch_size=32, epochs=rounds)
# model.fit(X_train, y_train, batch_size=32, epochs=nb_epoch,validation_split=0.2,callbacks=[history, early_stopping])
# model.fit(X_train, y_train, batch_size=32, epochs=30,validation_data=(X_val,y_val),callbacks=[history])
res0=model.predict(X_test)

In [ ]:
rounds=37
i=1
train=skf[i][0]
input_tensor = Input(X_train.shape[1:])
x = input_tensor
x = Dropout(dout)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(classes, activation='softmax')(x)
model = Model(inputs=input_tensor, outputs=x)
model.compile(optimizer='adadelta',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
print("Fold", i)
model.fit(X_train[train], y_train[train], batch_size=32, epochs=rounds)
res1=model.predict(X_test)

In [ ]:
rounds=37
i=2
train=skf[i][0]
input_tensor = Input(X_train.shape[1:])
x = input_tensor
x = Dropout(dout)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(classes, activation='softmax')(x)
model = Model(inputs=input_tensor, outputs=x)
model.compile(optimizer='adadelta',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
print("Fold", i)
model.fit(X_train[train], y_train[train], batch_size=32, epochs=rounds)
res2=model.predict(X_test)

In [ ]:
rounds=37
i=3
train=skf[i][0]
input_tensor = Input(X_train.shape[1:])
x = input_tensor
x = Dropout(dout)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(classes, activation='softmax')(x)
model = Model(inputs=input_tensor, outputs=x)
model.compile(optimizer='adadelta',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
print("Fold", i)
model.fit(X_train[train], y_train[train], batch_size=32, epochs=rounds)
res3=model.predict(X_test)

In [ ]:
rounds=37
i=4
train=skf[i][0]
input_tensor = Input(X_train.shape[1:])
x = input_tensor
x = Dropout(dout)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(classes, activation='softmax')(x)
model = Model(inputs=input_tensor, outputs=x)
model.compile(optimizer='adadelta',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
print("Fold", i)
model.fit(X_train[train], y_train[train], batch_size=32, epochs=rounds)
res4=model.predict(X_test)

In [ ]:
res0.argmax(axis=1)

In [ ]:
with h5py.File("CV-final\\datax-r152_d161_x-37softmax.h5") as h:
    h.create_dataset("prob0", data=res0)
    h.create_dataset("prob1", data=res1)
    h.create_dataset("prob2", data=res2)
    h.create_dataset("prob3", data=res3)
    h.create_dataset("prob4", data=res4)